In [ ]:
!pip install foolbox

In [2]:
import torch
import eagerpy as ep
from foolbox import PyTorchModel, accuracy, samples
import foolbox.attacks as fa
import numpy as np

     |████████████████████████████████| 1.7 MB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 159 kB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 2.2 MB/s eta 0:00:01
  Attempting uninstall: requests
    Found existing installation: requests 2.22.0
    Uninstalling requests-2.22.0:
      Successfully uninstalled requests-2.22.0


In [12]:
bottlenecks = [1,2,4,8,16,32]
runs = range(0,10)
ventraldepths = [0,1,2,3,4]

attacks = [
    fa.FGSM(),
    fa.LinfPGD(),
    fa.LinfBasicIterativeAttack(),
    fa.LinfAdditiveUniformNoiseAttack(),
    fa.LinfDeepFoolAttack(),
]

epsilons = [
    0.0,
    0.0005,
    0.001,
    0.0015,
    0.002,
    0.003,
    0.005,
    0.01,
    0.02,
    0.03,
    0.1,
    0.3,
    0.5,
    1.0,
]

results = dict()

for vdepth in ventraldepths:
    results[vdepth] = dict()
    for bn in bottlenecks:
        results[vdepth][bn] = dict()
        for run in runs:
            print(vdepth, bn, run)
            
            results[vdepth][bn][run] = dict()
            
            model = torch.hub.load('ecs-vlc/opponency:master', 'colour_full', n_bn=bn, d_vvs=vdepth, rep=run)
            model.eval()
            fmodel = PyTorchModel(model, bounds=(0, 1))

            images, labels = samples(fmodel, dataset="cifar10", batchsize=20)
            images = images.contiguous()

            # get data and test the model
            # wrapping the tensors with ep.astensors is optional, but it allows
            # us to work with EagerPy tensors in the following
            results[vdepth][bn][run]["accuracy"] = accuracy(fmodel, images, labels)

            attack_success = np.zeros((len(attacks), len(epsilons), len(images)), dtype=np.bool)
            for i, attack in enumerate(attacks):
                _, _, success = attack(fmodel, images, labels, epsilons=epsilons)
                success_ = success.cpu().numpy()
                attack_success[i] = success_
                results[vdepth][bn][run][str(attack)] = 1.0 - success_.mean(axis=-1)

            robust_accuracy = 1.0 - attack_success.max(axis=0).mean(axis=-1)
            results[vdepth][bn][run]['robust_accuracy'] = robust_accuracy

Downloading: "https://github.com/ecs-vlc/opponency/archive/master.zip" to /Users/jsh2/.cache/torch/hub/master.zip
Downloading: "http://marc.ecs.soton.ac.uk/pytorch-models/opponency/cifar10/colour_full/model_2_2_0.pt" to /Users/jsh2/.cache/torch/checkpoints/model_2_2_0.pt
100%|██████████| 128M/128M [00:56<00:00, 2.40MB/s] 


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.